# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [73]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [74]:
data_df = pd.read_csv("../output_data/cities.csv")
data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kamaishi,39.2667,141.8833,79.43,78,22,3.31,JP,1626851324
1,São João da Barra,-21.6403,-41.0511,59.45,76,66,6.93,BR,1626851325
2,Nantucket,41.2835,-70.0995,72.01,96,1,5.75,US,1626851325
3,Avarua,-21.2078,-159.7750,75.25,88,19,8.05,CK,1626851325
4,Ponta Delgada,37.7333,-25.6667,68.72,83,40,5.75,PT,1626851326
...,...,...,...,...,...,...,...,...,...
555,Matamoros,25.5333,-103.2500,76.10,58,10,11.07,MX,1626851538
556,Ilulissat,69.2167,-51.1000,44.62,70,40,2.30,GL,1626851539
557,Cam Ranh,11.9214,109.1591,94.96,49,40,8.05,VN,1626851539
558,Stromness,58.9650,-3.2960,55.65,89,100,3.18,GB,1626851539


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [85]:
locations = data_df[["Lat", "Lng"]]

humidity = data_df["Humidity"].astype(float)

In [87]:
fig = gmaps.figure()

heatmap = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = humidity.max(), point_radius = 1)

fig.add_layer(heatmap)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

* A max temperature lower than 80 degrees but higher than 70.
* Wind speed less than 10 mph.
* Zero cloudiness.
* Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

In [77]:
new_df = data_df.loc[(data_df["Max Temp"] <= 80) & (data_df["Max Temp"] >= 70) & 
                     (data_df["Wind Speed"] <=10) & (data_df["Cloudiness"] == 0)]

new_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
45,Sola,-13.8833,167.5500,79.61,78,0,9.95,VU,1626851342
62,Margate,51.3813,1.3862,72.95,78,0,7.74,GB,1626851349
142,Piaçabuçu,-10.4056,-36.4344,73.47,79,0,9.82,BR,1626851378
179,Mocuba,-16.8392,36.9856,75.94,58,0,4.00,MZ,1626851393
185,Marsala,37.7966,12.4352,78.93,71,0,7.05,IT,1626851396
238,Algiers,36.7525,3.0420,71.42,73,0,2.30,DZ,1626851417
247,Jucurutu,-6.0339,-37.0203,72.23,73,0,9.69,BR,1626851420
300,Shelabolikha,53.4131,82.6167,76.46,37,0,9.93,RU,1626851441
308,Requena,39.4883,-1.1004,73.00,45,0,1.66,ES,1626851444
357,Tiznit Province,29.5833,-9.5000,78.75,39,0,5.08,MA,1626851462


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [78]:
hotel_df = new_df.loc[:, ('City', 'Country', 'Lat', 'Lng') ].reset_index(drop = True)

hotel_df["Hotel Name"] = ""

hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
0,Sola,VU,-13.8833,167.5500,
1,Margate,GB,51.3813,1.3862,
2,Piaçabuçu,BR,-10.4056,-36.4344,
3,Mocuba,MZ,-16.8392,36.9856,
4,Marsala,IT,37.7966,12.4352,


In [79]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "",
    "radius": 5000,
    "type": "hotel",
    "keyword": "hotel",
    "key": g_key,
}

for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    name = row["City"]
    params["location"] = f"{lat},{lng}"

    response = requests.get(base_url, params=params).json()
    print(f"Searching for city number {index + 1}: {name}")
    
    
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        print(f"Closest hotel in {name} is: {response['results'][0]['name']}")
        print("------------------")

    except (KeyError, IndexError):
        print(f"Missing result for City: {name}.")
        print("------------------")
        

print("-----------------------------")
print("Data Retrieval Complete")
print("-----------------------------")    

Searching for city number 1: Sola
Missing result for City: Sola.
------------------
Searching for city number 2: Margate
Closest hotel in Margate is: The Albion Rooms
------------------
Searching for city number 3: Piaçabuçu
Closest hotel in Piaçabuçu is: Hotel Cara Peba
------------------
Searching for city number 4: Mocuba
Closest hotel in Mocuba is: Aschel guest house
------------------
Searching for city number 5: Marsala
Closest hotel in Marsala is: Grand Hotel Palace
------------------
Searching for city number 6: Algiers
Closest hotel in Algiers is: Hotel Aurassi
------------------
Searching for city number 7: Jucurutu
Closest hotel in Jucurutu is: Pousada Nossa Casa
------------------
Searching for city number 8: Shelabolikha
Closest hotel in Shelabolikha is: Gostinitsa Uyut
------------------
Searching for city number 9: Requena
Closest hotel in Requena is: Hotel Doña Anita
------------------
Searching for city number 10: Tiznit Province
Missing result for City: Tiznit Provinc

In [80]:
hotel_df = hotel_df.loc[(hotel_df["Hotel Name"] != ""), :]
hotel_df
#hotel_df = hotel_df[["Hotel Name", "City", "Country", "Lat", "Lng"]]



,City,Country,Lat,Lng,Hotel Name
1,Margate,GB,51.3813,1.3862,The Albion Rooms
2,Piaçabuçu,BR,-10.4056,-36.4344,Hotel Cara Peba
3,Mocuba,MZ,-16.8392,36.9856,Aschel guest house
4,Marsala,IT,37.7966,12.4352,Grand Hotel Palace
5,Algiers,DZ,36.7525,3.0420,Hotel Aurassi
6,Jucurutu,BR,-6.0339,-37.0203,Pousada Nossa Casa
7,Shelabolikha,RU,53.4131,82.6167,Gostinitsa Uyut
8,Requena,ES,39.4883,-1.1004,Hotel Doña Anita
13,Ponta do Sol,PT,32.6667,-17.1000,Hotel Enotel Baía
14,Mitsamiouli,KM,-11.3847,43.2844,Al Camar Lodge


In [88]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [89]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)

fig
# Display figure


Figure(layout=FigureLayout(height='420px'))